# Load Denmark COVID-19 cases by age group

### Andrei Paleyes
### May 6, 2020

In Denmark, official COVID-19 data is provided daily by Statens Serum Institut, SSI. Here is the reference web page: https://www.ssi.dk/aktuelt/sygdomsudbrud/coronavirus/covid-19-i-danmark-epidemiologisk-overvaagningsrapport

Age-stratified data is available in daily reports that are provided as PDF files. Each of them (except one so far) contain a single table that is of interest to us, that has cumulative number of COVID-19 cases for male and female in each age group (each group is a 10 year step).

Unfortunately at the time of writing this notebook there is no way to consume this data in machine-friendly format. Therefore this notebook attempts to parse the daily report PDF files and compile the dataset.

In [1]:
!java -version

java version "12.0.1" 2019-04-16
Java(TM) SE Runtime Environment (build 12.0.1+12)
Java HotSpot(TM) 64-Bit Server VM (build 12.0.1+12, mixed mode, sharing)


In [2]:
%pip install -q tabula-py

     |████████████████████████████████| 10.4MB 8.0MB/s 


In [2]:
import tabula

tabula.environment_info()

Python version:
    3.7.3 (default, Mar 27 2019, 16:54:48) 
[Clang 4.0.1 (tags/RELEASE_401/final)]
Java version:
    java version "12.0.1" 2019-04-16
Java(TM) SE Runtime Environment (build 12.0.1+12)
Java HotSpot(TM) 64-Bit Server VM (build 12.0.1+12, mixed mode, sharing)
tabula-py version: 2.1.0
platform: Darwin-19.2.0-x86_64-i386-64bit
uname:
    uname_result(system='Darwin', node='Admins-MacBook-Pro.local', release='19.2.0', version='Darwin Kernel Version 19.2.0: Sat Nov  9 03:47:04 PST 2019; root:xnu-6153.61.1~20/RELEASE_X86_64', machine='x86_64', processor='i386')
linux_distribution: ('Darwin', '19.2.0', '')
mac_ver: ('10.15.2', ('', '', ''), 'x86_64')
    


In [3]:
import pandas as pd

In [0]:
# path: the url of the report
# page: page in the document where the necessary table is found
# top: distance from the top of the page to the top of the table. defaults to 110 pt.
# bottom: distance from the top of the page to the bottom of the table. Defaults to 350 + (top - 110) pt.
#
# More details on how to measure top and bottom: https://stackoverflow.com/questions/45457054/tabula-extract-tables-by-area-coordinates

all_reports = {
    'May 6, 2020': {'path': "https://files.ssi.dk/COVID19-overvaagningsrapport-06052020-s0l0", 'page': 11},
    'May 5, 2020': {'path': "https://files.ssi.dk/COVID19-overvaagningsrapport-05052020-s0l0", 'page': 11},
    'May 4, 2020': {'path': "https://files.ssi.dk/COVID19-overvaagningsrapport-04052020-hu28", 'page': 11},
    'May 3, 2020': {'path': "https://files.ssi.dk/COVID19-overvaagningsrapport-03052020-am43", 'page': 11},
    'May 2, 2020': {'path': "https://files.ssi.dk/COVID19-overvaagningsrapport-02052020-l9i8", 'page': 11},
    'May 1, 2020': {'path': "https://files.ssi.dk/COVID19-overvaagningsrapport-01052020-prst",  'page': 11},
    'April 30, 2020': {'path': "https://files.ssi.dk/COVID19-overvaagningsrapport-30042020-2h7d",   'page': 11},
    'April 29, 2020': {'path': "https://files.ssi.dk/COVID19-overvaagningsrapport-29042020-wl02",   'page': 11},
    'April 28, 2020': {'path': "https://files.ssi.dk/COVID19-overvaagningsrapport-28042020-gg64",   'page': 11},
    'April 27, 2020': {'path': "https://files.ssi.dk/COVID19-overvaagningsrapport-27042020-ce23",   'page': 11},
    'April 26, 2020': {'path': "https://files.ssi.dk/COVID19-overvaagningsrapport-26042020-y34f",   'page': 11},
    'April 25, 2020': {'path': "https://files.ssi.dk/COVID19-overvaagningsrapport-25042020-sr21",   'page': 11},
    'April 24, 2020': {'path': "https://files.ssi.dk/COVID19-overvaagningsrapport-24042020-ds65",   'page': 11},
    'April 23, 2020': {'path': "https://files.ssi.dk/COVID19-overvaagningsrapport-23042020-gl5b",  'page': 11},
    'April 22, 2020': {'path': "https://files.ssi.dk/covid19-overvaagningsrapport-22042020-lj45",  'page': 11},
    'April 21, 2020': {'path': "https://files.ssi.dk/covid19-overvaagningsrapport-21032020-hj78",   'page': 11},
    'April 20, 2020': {'path': "https://files.ssi.dk/COVID19-overvaagningsrapport-20042020-2dd09",  'page': 11},
    'April 19, 2020': {'path': "https://files.ssi.dk/COVID19-overvaagningsrapport-19042020-hba7",  'page': 11},
    'April 18, 2020': {'path': "https://files.ssi.dk/COVID19-overvaagningsrapport-18042020-a8sg",   'page': 11},
    'April 17, 2020': {'path': "https://www.ssi.dk/-/media/arkiv/dk/aktuelt/sygdomsudbrud/covid19-rapport/17042020/covid19-overvaagningsrapport-17042020-gt90.pdf",   'page': 11},
    'April 16, 2020': {'path': "https://files.ssi.dk/COVID19-overvaagningsrapport-16042020-hzz5",   'page': 11, 'top':170},
    'April 15, 2020': {'path': "https://files.ssi.dk/COVID19-overvaagningsrapport-15042020-ht7b",   'page': 11, 'top':170},
    'April 14, 2020': {'path': "https://files.ssi.dk/COVID19-overvaagningsrapport-14042020-wgkv",   'page': 10, 'top':170},
    'April 13, 2020': {'path': "https://files.ssi.dk/COVID19-overvaagningsrapport-13042020-gy70",   'page': 10, 'top':170},
    'April 12, 2020': {'path': "https://files.ssi.dk/COVID19-overvaagningsrapport-12042020-hh8b",   'page': 10, 'top':170},
    'April 11, 2020': {'path': "https://files.ssi.dk/COVID19-overvaagningsrapport-11042020-ednk",   'page': 10, 'top':170},
    'April 10, 2020': {'path': "https://files.ssi.dk/COVID19-overvaagningsrapport-10042020-21bn",   'page': 10, 'top':170},
    'April 9, 2020': {'path': "https://files.ssi.dk/COVID19-overvaagningsrapport-09042020-31us",  'page': 10, 'top':125},
    'April 8, 2020': {'path': "https://files.ssi.dk/COVID19-overvaagningsrapport-08042020-zm92",  'page': 10, 'top':125},
    'April 7, 2020': {'path': "https://files.ssi.dk/COVID19-overvaagningsrapport-07042020-wvp1",  'page': 10, 'top':150},
    'April 6, 2020': {'path': "https://files.ssi.dk/covid19-overvaagningsrapport-06042020-hu4v", 'page': 10, 'top':150},
    'April 5, 2020': {'path': "https://files.ssi.dk/COVID19-overvaagningsrapport-05042020-dd29", 'page': 10, 'top':150}, 
    'April 4, 2020': {'path': "https://files.ssi.dk/COVID19-overvaagningsrapport-04042020-wdcs",  'page': 10, 'top':150},
    'April 3, 2020': {'path': "https://files.ssi.dk/covid-19-overvaagningsrapport-03042020-2",    'page': 10, 'top':150},
    'April 2, 2020': {'path': "https://files.ssi.dk/COVID19-overvaagningsrapport-02042020-kl45",  'page': 9, 'top':150},
    'April 1, 2020': {'path': "https://files.ssi.dk/COVID19-overvaagningsrapport-01042020-apl4",  'page': 9, 'top':170},
    'March 31, 2020': {'path': "https://files.ssi.dk/COVID19-overvaagningsrapport-31032020-2us61",   'page': 9, 'top':150},
    'March 30, 2020': {'path': "https://files.ssi.dk/COVID19-overvaagningsrapport-30032020-hb2a",   'page': 9, 'top':150},
    'March 29, 2020': {'path': "https://files.ssi.dk/COVID19-overvaagningsrapport-29032020-f67s",   'page': 9, 'top':150},
    'March 28, 2020': {'path': "https://files.ssi.dk/COVID19-overvaagningsrapport-28032020-wl35",   'page': 9, 'top':150},
    'March 27, 2020': {'path': "https://files.ssi.dk/COVID19-overvaagningsrapport-27032020-gk38",  'page': 9, 'top':150},
    'March 26, 2020': {'path': "https://files.ssi.dk/COVID19-overvaagningsrapport-26032020",   'page': 9, 'top':150},
    'March 25, 2020': {'path': "https://files.ssi.dk/COVID19-overvaagningsrapport-25032020",   'page': 9, 'top':150},
    'March 24, 2020': {'path': "https://files.ssi.dk/COVID19-overvaagningsrapport-24032020",  'page': 9, 'top':150},
    'March 23, 2020': {'path': "https://files.ssi.dk/COVID19-overvaagningsrapport-23032020",   'page': 5, 'top':135, 'bottom': 285},
    'March 22, 2020': {'path': "https://files.ssi.dk/COVID19-overvaagningsrapport-22032020",   'page': 5, 'top':135, 'bottom': 285},
    'March 21, 2020': {'path': "https://files.ssi.dk/COVID19-overvaagningsrapport-21032020-2",   'page': 5, 'top':150, 'bottom': 300},
    'March 20, 2020': {'path': "https://files.ssi.dk/COVID19-overvaagningsrapport-20032020",   'page': 5, 'top':160, 'bottom': 310},
    'March 19, 2020': {'path': "https://files.ssi.dk/COVID19-overvaagningsrapport-19032020",   'page': 5, 'top':135, 'bottom': 285},
    'March 18, 2020': {'path': "https://files.ssi.dk/COVID19-overvaagningsrapport-18032020",   'page': 4, 'top':515, 'bottom': 670},
    'March 17, 2020': {'path': "https://files.ssi.dk/COVID19-overvaagningsrapport-17032020",   'page': 4, 'top':515, 'bottom': 670},
    'March 16, 2020': {'path': "https://files.ssi.dk/COVID19-overvaagningsrapport-16032020",   'page': 3, 'top':545, 'bottom': 700},
    'March 13, 2020': {'path': "https://files.ssi.dk/COVID19-overvaagningsrapport-13032020",   'page': 3, 'top':135, 'bottom': 285},
    # this one does not contain necessary info at all
    #'March 12, 2020': {'path': "https://files.ssi.dk/COVID19-overvaagningsrapport-12032020", 'page': 3, 'top':160, 'bottom': 310}
}

In [6]:
df = pd.DataFrame(index=range(10))
for date in all_reports:
  data_link = all_reports[date]
  top = data_link.get('top', 110)
  bottom = data_link.get('bottom', 350 + (top - 110))
  report_df = tabula.read_pdf(data_link['path'], pages=data_link['page'], guess=False, area=[top, 0.0, bottom, 550], pandas_options={'dtype': 'str'}, multiple_tables=False)[0]
  df[date] = report_df.head(10).iloc[:,-1]
  print('Processed ' + str(date))

Processed May 5, 2020
Processed May 4, 2020
Processed May 3, 2020
Processed May 2, 2020
Processed May 1, 2020
Processed April 30, 2020
Processed April 29, 2020
Processed April 28, 2020
Processed April 27, 2020
Processed April 26, 2020
Processed April 25, 2020
Processed April 24, 2020
Processed April 23, 2020
Processed April 22, 2020
Processed April 21, 2020
Processed April 20, 2020
Processed April 19, 2020
Processed April 18, 2020
Processed April 17, 2020
Processed April 16, 2020
Processed April 15, 2020
Processed April 14, 2020
Processed April 13, 2020
Processed April 12, 2020
Processed April 11, 2020
Processed April 10, 2020
Processed April 9, 2020
Processed April 8, 2020
Processed April 7, 2020
Processed April 6, 2020
Processed April 5, 2020
Processed April 4, 2020
Processed April 3, 2020
Processed April 2, 2020
Processed April 1, 2020
Processed March 31, 2020
Processed March 30, 2020
Processed March 29, 2020
Processed March 28, 2020
Processed March 27, 2020
Processed March 26, 2020

Got stderr: May 06, 2020 10:00:47 PM org.apache.pdfbox.pdmodel.font.FileSystemFontProvider loadDiskCache
May 06, 2020 10:00:47 PM org.apache.pdfbox.pdmodel.font.FileSystemFontProvider <init>
May 06, 2020 10:00:47 PM org.apache.pdfbox.pdmodel.font.FileSystemFontProvider <init>



Processed March 23, 2020
Processed March 22, 2020
Processed March 21, 2020
Processed March 20, 2020
Processed March 19, 2020
Processed March 18, 2020
Processed March 17, 2020
Processed March 16, 2020
Processed March 13, 2020


In [7]:
df

,"May 5, 2020","May 4, 2020","May 3, 2020","May 2, 2020","May 1, 2020","April 30, 2020","April 29, 2020","April 28, 2020","April 27, 2020","April 26, 2020","April 25, 2020","April 24, 2020","April 23, 2020","April 22, 2020","April 21, 2020","April 20, 2020","April 19, 2020","April 18, 2020","April 17, 2020","April 16, 2020","April 15, 2020","April 14, 2020","April 13, 2020","April 12, 2020","April 11, 2020","April 10, 2020","April 9, 2020","April 8, 2020","April 7, 2020","April 6, 2020","April 5, 2020","April 4, 2020","April 3, 2020","April 2, 2020","April 1, 2020","March 31, 2020","March 30, 2020","March 29, 2020","March 28, 2020","March 27, 2020","March 26, 2020","March 25, 2020","March 24, 2020","March 23, 2020","March 22, 2020","March 21, 2020","March 20, 2020","March 19, 2020","March 18, 2020","March 17, 2020","March 16, 2020","March 13, 2020"
0,168,155,143,139,135,126,120,116,111,108,104,95,93,89,87,84,82,81,80,75,72,69,66,63,63,60,58,47,38,34,32,27,23,21,21,19,18,17,15,15,13,13,13,13,13,13,13,13,12,13,12,10
1,378,363,347,345,336,326,313,298,284,273,266,252,244,234,224,215,209,198,187,181,172,167,162,156,150,135,129,114,102,90,82,74,63,53,48,48,46,42,41,40,40,40,37,35,35,35,35,35,33,33,33,30
2,1.308,1.287,1.265,1.244,1.227,1.188,1.160,1.139,1.112,1.093,1.064,1.031,1.011,986,960,932,903,878,860,826,796,763,738,706,678,640,607,582,510,466,423,386,328,300,276,255,237,222,206,196,188,179,172,167,165,161,153,148,146,142,140,134
3,1.368,1.339,1.315,1.302,1.284,1.262,1.230,1.199,1.181,1.162,1.140,1.113,1.095,1.075,1.046,1.014,992,971,945,920,898,877,846,823,788,763,745,708,643,594,545,492,433,394,355,315,289,279,260,239,223,211,203,193,191,184,170,162,156,147,143,135
4,1.778,1.757,1.734,1.719,1.706,1.687,1.673,1.644,1.616,1.600,1.579,1.545,1.512,1.488,1.448,1.419,1.406,1.381,1.353,1.315,1.277,1.243,1.212,1.188,1.165,1.140,1.105,1.062,1.002,949,890,837,765,711,660,598,544,520,486,454,436,420,402,386,375,362,345,325,305,294,284,253
5,1.833,1.805,1.780,1.755,1.733,1.714,1.694,1.682,1.656,1.637,1.620,1.588,1.565,1.537,1.491,1.457,1.441,1.420,1.393,1.369,1.317,1.278,1.251,1.220,1.187,1.155,1.125,1.073,989,917,869,814,733,660,599,536,482,447,409,365,331,309,279,254,248,239,224,209,200,186,176,159
6,1.178,1.164,1.155,1.141,1.137,1.122,1.106,1.090,1.074,1.068,1.054,1.028,1.017,998,971,949,939,919,897,872,853,833,812,798,781,763,738,711,656,616,588,546,486,436,408,369,330,306,267,250,228,204,180,156,148,140,127,107,90,70,62,50
7,876,871,864,851,848,842,833,825,819,806,801,776,766,751,736,728,716,708,696,681,674,664,647,642,628,620,607,586,560,534,512,490,462,436,409,388,355,331,304,261,224,192,167,132,118,100,82,70,55,39,26,5
8,706,701,693,685,682,673,666,649,642,628,623,593,588,577,556,545,531,524,507,493,482,468,454,448,432,422,409,395,376,355,342,331,310,286,260,240,214,194,178,158,140,124,106,98,87,77,65,51,38,28,18,7
9,228,228,227,226,223,218,213,209,203,200,194,189,182,177,176,172,165,162,155,147,140,134,130,130,124,121,112,108,102,92,86,80,69,58,56,47,40,37,35,32,28,23,18,16,15,15,12,12,9,8,4,2


In [8]:
import numpy as np

# avoid all weirdness of commas and dots as delimiters
df = df.astype('string').apply(lambda x: x.str.replace('.','')).astype(int)

# set index to age groups
df = df.set_index(pd.Index([f'{i}-{i+9}' for i in range(0, 90, 10)] + ['90+'], dtype='str'))
df.columns = pd.to_datetime(df.columns.astype(str))
df = df[np.sort(df.columns)]
df

,2020-03-13,2020-03-16,2020-03-17,2020-03-18,2020-03-19,2020-03-20,2020-03-21,2020-03-22,2020-03-23,2020-03-24,2020-03-25,2020-03-26,2020-03-27,2020-03-28,2020-03-29,2020-03-30,2020-03-31,2020-04-01,2020-04-02,2020-04-03,2020-04-04,2020-04-05,2020-04-06,2020-04-07,2020-04-08,2020-04-09,2020-04-10,2020-04-11,2020-04-12,2020-04-13,2020-04-14,2020-04-15,2020-04-16,2020-04-17,2020-04-18,2020-04-19,2020-04-20,2020-04-21,2020-04-22,2020-04-23,2020-04-24,2020-04-25,2020-04-26,2020-04-27,2020-04-28,2020-04-29,2020-04-30,2020-05-01,2020-05-02,2020-05-03,2020-05-04,2020-05-05
0-9,10,12,13,12,13,13,13,13,13,13,13,13,15,15,17,18,19,21,21,23,27,32,34,38,47,58,60,63,63,66,69,72,75,80,81,82,84,87,89,93,95,104,108,111,116,120,126,135,139,143,155,168
10-19,30,33,33,33,35,35,35,35,35,37,40,40,40,41,42,46,48,48,53,63,74,82,90,102,114,129,135,150,156,162,167,172,181,187,198,209,215,224,234,244,252,266,273,284,298,313,326,336,345,347,363,378
20-29,134,140,142,146,148,153,161,165,167,172,179,188,196,206,222,237,255,276,300,328,386,423,466,510,582,607,640,678,706,738,763,796,826,860,878,903,932,960,986,1011,1031,1064,1093,1112,1139,1160,1188,1227,1244,1265,1287,1308
30-39,135,143,147,156,162,170,184,191,193,203,211,223,239,260,279,289,315,355,394,433,492,545,594,643,708,745,763,788,823,846,877,898,920,945,971,992,1014,1046,1075,1095,1113,1140,1162,1181,1199,1230,1262,1284,1302,1315,1339,1368
40-49,253,284,294,305,325,345,362,375,386,402,420,436,454,486,520,544,598,660,711,765,837,890,949,1002,1062,1105,1140,1165,1188,1212,1243,1277,1315,1353,1381,1406,1419,1448,1488,1512,1545,1579,1600,1616,1644,1673,1687,1706,1719,1734,1757,1778
50-59,159,176,186,200,209,224,239,248,254,279,309,331,365,409,447,482,536,599,660,733,814,869,917,989,1073,1125,1155,1187,1220,1251,1278,1317,1369,1393,1420,1441,1457,1491,1537,1565,1588,1620,1637,1656,1682,1694,1714,1733,1755,1780,1805,1833
60-69,50,62,70,90,107,127,140,148,156,180,204,228,250,267,306,330,369,408,436,486,546,588,616,656,711,738,763,781,798,812,833,853,872,897,919,939,949,971,998,1017,1028,1054,1068,1074,1090,1106,1122,1137,1141,1155,1164,1178
70-79,5,26,39,55,70,82,100,118,132,167,192,224,261,304,331,355,388,409,436,462,490,512,534,560,586,607,620,628,642,647,664,674,681,696,708,716,728,736,751,766,776,801,806,819,825,833,842,848,851,864,871,876
80-89,7,18,28,38,51,65,77,87,98,106,124,140,158,178,194,214,240,260,286,310,331,342,355,376,395,409,422,432,448,454,468,482,493,507,524,531,545,556,577,588,593,623,628,642,649,666,673,682,685,693,701,706
90+,2,4,8,9,12,12,15,15,16,18,23,28,32,35,37,40,47,56,58,69,80,86,92,102,108,112,121,124,130,130,134,140,147,155,162,165,172,176,177,182,189,194,200,203,209,213,218,223,226,227,228,228


In [0]:
df.to_csv('denmark_cases_by_age.csv') 

In [0]:
# only if in colab
from google.colab import files
files.download('denmark_cases_by_age.csv')